In [6]:
import json

import pandas as pd
import numpy as np

In [3]:
with open(r'./lyrics_emotion[0-5250].json', 'r') as fn:
    emo_dict = json.load(fn)

In [8]:
df = pd.read_csv(r'./represented_lyrics.csv')
df.head()

,index,song_x,year_x,artist_x,genre,lyrics_x,word_count,represent_x,00,000,...,ä½,å¹,å¹³é,åº,æ³,çª,ç¹,ç½,ç½ªã,éº
0,0,louder-flux-pavilion-doctor-p-remix,2012,dj-fresh,Electronic,I can't control this feeling Something's happe...,387,I ca n't control feel someth 's happen insid o...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,that-s-my-name,2009,akcent,Pop,In my heart I will keep you In my heart Foreve...,191,In heart I keep In heart forev In heart and on...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,lemonade,2007,apologetix,Rock,"Oh, Mama, I've been cleared of my crimes and I...",157,Oh mama I 've clear crime I 'm law law put end...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,return-of-the-hustle,2007,fabolous,Hip-Hop,Record mode! (Record mode!) Get your money in ...,497,record mode record mode get money air like yea...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,change-the-world,2013,calinator,Not Available,Intro Calinator!I dont know what youre doing b...,313,intro calin I dont know your Im feel right cho...,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
